In [1]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

from PIL import Image
import requests

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?painting ?paintingLabel ?artist ?artistLabel ?originLabel ?image WHERE {
  ?painting wdt:P31 wd:Q3305213;
    wdt:P170 ?artist;
    wdt:P276 ?origin.
  ?artist wdt:P27 wd:Q38.
  ?origin wdt:P17 wd:Q38.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
  OPTIONAL { ?painting wdt:P18 ?image. }
}
LIMIT 9"""

# SUBROUTINES

def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

def get_image(url, headers):
    r = requests.get(url, headers=headers, stream=True)
    if r.status_code == 200:
        im = Image.open(r.raw)
        return im
    if r.status_code == 500:
        return None
    if r.status_code == 403:
        return None
    if r.status_code == 429:
        timeout = get_delay(r.headers['retry-after'])
        print('Timeout {} m {} s'.format(timeout // 60, timeout % 60))
        time.sleep(timeout)
        get_image(url, headers)

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# MAIN PROGRAM

results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print('Wikidata link: ' + '[' + result['painting']['value'] + ']' + '(' + result['painting']['value'] + ')' + '\n')
    print('Title: ' + result['paintingLabel']['value'] + '\n')
    print('Creator: ' + result['artistLabel']['value'] + '\n')

    if 'image' in result:
      # get image from image URL and display resized version
      image_url=result['image']['value']
      headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
      im = get_image(image_url, headers)
      im.thumbnail((500, 500), Image.Resampling.LANCZOS)
      display(im)
    print('\n\n')


Wikidata link: [http://www.wikidata.org/entity/Q28803748](http://www.wikidata.org/entity/Q28803748)

Title: Q28803748

Creator: Giovanni Benedetto Castiglione



AttributeError: module 'PIL.Image' has no attribute 'Resampling'